**Práctica Integrada: Pipeline de Preprocesamiento para Análisis de Feedback en E-Commerce**  
**Contexto Empresarial Real:**  
Eres parte del equipo de NLP de **MarketMind**, una empresa de e-commerce que analiza millones de comentarios de clientes (inglés/español) y tweets para detectar tendencias de productos. **Problema actual:** El modelo GPT-4 fine-tuned genera resúmenes erróneos porque el texto crudo contiene URLs, precios, y emoticones mal normalizados.  

---

### **Fase 1: Limpieza Contextual de Texto**  
**Objetivo:** Crear una función que procese texto crudo preservando información crítica para análisis comercial.  

#### **Datos de Entrada:**  

Archivo -> validation_samples.json

#### **Requisitos:**  
1. **Eliminar:**  
   - URLs, handles de usuario (@MariaP), y signos de puntuación *excepto* `!`, `?`, `%`, `$`, `/`.  
   - Paréntesis `()` y corchetes `[]`.  
2. **Preservar:**  
   - Emoticones (😃, 🔥), hashtags (#ModaDeportiva2023), y formatos comerciales (`2x`, `$99.99`).  
   - Fechas (`30/11/2023`), horas (`18:30`), y porcentajes (`15%`).  
3. **Normalizar:**  
   - Unificar espacios múltiples y saltos de línea.  

#### **Pistas de Implementación:**  
- Usar regex con grupos capturadores para fechas (`\b\d{2}/\d{2}/\d{4}\b`).  
- Para emoticones, usar la librería `emoji` (no eliminar 🚫→ ✅).  
- Preservar `$`, `%`, y `/` solo si están adyacentes a números: `\$?\d+(\.\d+)?%?`.  

#### **Verificación:**  
El texto procesado debe quedar:  
```  
🔥 OFERTA Compre 2x zapatos Nike a $99.99 antes $150 👟  
Válido hasta el 30/11/2023 Atención ¿Envío gratis 😃 #ModaDeportiva2023  
```  

In [1]:
import re
import json
import emoji
import random

def clean_text(text):
    """Función de limpieza de texto para análisis de feedback en e-commerce.
    
    Parámetros:
    text (str): Texto a limpiar
    
    Returns:
    str: Texto limpio y normalizado
    """
    # 1. Identificar y marcar elementos a preservar
    
    # Patrones para elementos a preservar
    patterns_to_preserve = [
        # Hashtags
        (r'(#\w+)', r' \1 '),
        
        # Fechas en formato DD/MM/YYYY o YYYY-MM-DD
        (r'(\b\d{2}/\d{2}/\d{4}\b|\b\d{4}-\d{2}-\d{2}\b)', r' \1 '),
        
        # Horas en formato HH:MM
        (r'(\b\d{1,2}:\d{2}\b)', r' \1 '),
        
        # Formatos comerciales: 2x, $99.99, 15%, etc.
        (r'(\b\d+x\b)', r' \1 '),  # Preservar formatos como 2x
        (r'(\$?\d+(\.\d+)?(€|USD)?)', r' \1 '),  # Preservar precios
        (r'(\d+(\.\d+)?%)', r' \1 ')  # Preservar porcentajes
    ]
    
    # Aplicar patrones de preservación
    for pattern, replacement in patterns_to_preserve:
        text = re.sub(pattern, replacement, text)
    
    # 2. Eliminar elementos no deseados
    
    # Eliminar URLs
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    
    # Eliminar handles de usuario (@usuario)
    text = re.sub(r'@\w+', ' ', text)
    
    # Eliminar paréntesis y corchetes pero preservar su contenido
    text = re.sub(r'[\(\)\[\]{}]', ' ', text)
    
    # 3. Crear una lista de caracteres a mantener (emojis y puntuación permitida)
    chars_to_keep = ['!', '?', '%', '$', '/', '€']
    
    # 4. Procesar el texto caracter por caracter
    result = []
    for char in text:
        # Mantener emojis
        if char in emoji.EMOJI_DATA:
            result.append(char)
        # Mantener puntuación permitida
        elif char in chars_to_keep:
            result.append(char)
        # Mantener letras, números y espacios
        elif char.isalnum() or char.isspace():
            result.append(char)
        # Reemplazar otra puntuación con espacio
        else:
            result.append(' ')
    
    # 5. Normalizar espacios múltiples y saltos de línea
    processed_text = ''.join(result)
    processed_text = re.sub(r'\s+', ' ', processed_text).strip()
    
    return processed_text

# Cargar los datos de ejemplo para pruebas
with open('validation_samples.json', 'r', encoding='utf-8') as f:
    samples = json.load(f)

# Seleccionar un ejemplo aleatorio para verificación
random_sample = random.choice(samples)
ejemplo = random_sample['input']

print("== Verificación Aleatoria ==")
print("\nTexto original:")
print(ejemplo)
print("\nTexto limpio:")
print(clean_text(ejemplo))

== Verificación Aleatoria ==

Texto original:
Best headphones ever! Bought for 129.95 USD on 11/11/2023. 🎧

Texto limpio:
Best headphones ever! Bought for 129 95 USD on 11 / 11 / 2023 🎧


---

### **Fase 2: Normalización de Números y Unidades**  
**Objetivo:** Reemplazar números genéricos pero preservar formatos clave para el modelo de precios.  

#### **Requisitos:**  
1. **Reemplazar:**  
   - Números sueltos (ej. `150` → `<NUM>`) excepto si están en fechas, precios, o unidades (`2x`).  
2. **Convertir:**  
   - Fechas a formato estándar ISO: `30/11/2023` → `2023-11-30`.  
   - Unidades de venta al por menor: `2x` → `2_unidades`, `3kg` → `3_kg`.  
3. **Procesar Monedas:**  
   - `$99.99` → `<USD>99.99`, `150€` → `<EUR>150`.  

#### **Pistas de Implementación:**  
- Usar `dateparser` para normalizar fechas en múltiples formatos (ej. `marzo 15, 2023` → `2023-03-15`).  
- Para unidades: `r'\b(\d+)(x|kg|ml)\b'` → `\1_\2`.  
- Diferenciar `$100` (precio) de `100$` (común en español) usando lookbehinds en regex.  

In [2]:
import dateparser
from datetime import datetime
import re

def normalize_text(text):
    """Función de normalización de números y unidades para análisis de feedback en e-commerce.
    
    Parámetros:
    text (str): Texto a normalizar (previamente limpiado)
    
    Returns:
    str: Texto normalizado
    """
    # 1. Normalizar fechas a formato ISO (YYYY-MM-DD)
    # Buscar fechas en formatos comunes (DD/MM/YYYY, YYYY-MM-DD, etc.)
    date_pattern = r'\b(\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{4}-\d{2}-\d{2})\b'
    
    def normalize_date(match):
        date_str = match.group(1)
        try:
            parsed_date = dateparser.parse(date_str)
            if parsed_date:
                return parsed_date.strftime('%Y-%m-%d')
            return date_str
        except:
            return date_str
    
    text = re.sub(date_pattern, lambda m: normalize_date(m), text)
    
    # 2. Normalizar unidades de venta (2x -> 2_unidades, 3kg -> 3_kg, etc.)
    # Patrones para diferentes tipos de unidades
    unit_patterns = [
        # Patrón para formatos como "2x"
        (r'\b(\d+)x\b', r'\1_unidades'),
        
        # Patrón para unidades de peso/volumen
        (r'\b(\d+)(kg|g|ml|l)\b', r'\1_\2')
    ]
    
    for pattern, replacement in unit_patterns:
        text = re.sub(pattern, replacement, text)
    
    # 3. Procesar monedas
    # USD: $99.99 -> <USD>99.99
    text = re.sub(r'\$\s*(\d+(?:\.\d+)?)', r'<USD>\1', text)
    
    # EUR: 150€ -> <EUR>150
    text = re.sub(r'(\d+(?:\.\d+)?)\s*€', r'<EUR>\1', text)
    text = re.sub(r'(\d+(?:\.\d+)?)\s*EUR', r'<EUR>\1', text)
    text = re.sub(r'(\d+(?:\.\d+)?)\s*euros?', r'<EUR>\1', text, flags=re.IGNORECASE)
    
    # USD texto: 99.99 USD -> <USD>99.99
    text = re.sub(r'(\d+(?:\.\d+)?)\s*USD', r'<USD>\1', text)
    
    # 4. Reemplazar números sueltos (que no sean parte de fechas, precios o unidades)
    # Primero identificamos patrones que NO deben ser reemplazados
    def replace_isolated_numbers(text):
        # Tokenizamos primero para tratar cada palabra
        tokens = text.split()
        result = []
        
        for token in tokens:
            # Saltar si ya está procesado como moneda, unidad o fecha
            if (re.match(r'<(USD|EUR)>\d+(\.\d+)?', token) or 
                re.match(r'\d+_\w+', token) or 
                re.match(r'\d{4}-\d{2}-\d{2}', token) or 
                re.match(r'#\w+', token)):
                result.append(token)
            # Reemplazar números sueltos
            elif token.isdigit() or re.match(r'^\d+\.\d+$', token):
                result.append('<NUM>')
            else:
                result.append(token)
                
        return ' '.join(result)
    
    text = replace_isolated_numbers(text)
    
    return text

# Procesamiento en pipeline: limpieza + normalización
def process_text(text):
    """Aplica el pipeline completo de procesamiento de texto: limpieza + normalización.
    
    Parámetros:
    text (str): Texto crudo a procesar
    
    Returns:
    str: Texto completamente limpio y normalizado
    """
    cleaned = clean_text(text)
    normalized = normalize_text(cleaned)
    return normalized

# Seleccionar un ejemplo aleatorio para verificación
random_sample = random.choice(samples)
ejemplo = random_sample['input']

print("== Verificación Pipeline Fase 2 ==\n")
print("Texto original:")
print(ejemplo)
print("\nTexto limpio:")
cleaned = clean_text(ejemplo)
print(cleaned)
print("\nTexto normalizado:")
print(normalize_text(cleaned))

# Ejemplo específico para verificar contra los requisitos
ejemplo_especifico = "🔥¡OFERTA! Compre 2x zapatos Nike a $99.99 (antes $150) 👟. ¡Válido hasta el 30/11/2023! Visita https://marketmind.com/oferta-nike. Atención @MariaP: ¿Envío gratis? 😃 #ModaDeportiva2023."
print("\n== Verificación con Ejemplo Específico ==\n")
print("Texto original:")
print(ejemplo_especifico)
print("\nTexto normalizado (pipeline completo):")
print(process_text(ejemplo_especifico))

== Verificación Pipeline Fase 2 ==

Texto original:
I love my new iPhone 12! 😍 Battery life is amazing. Bought it at 799€ from https://apple.com. #TechReview

Texto limpio:
I love my new iPhone 12 ! 😍 Battery life is amazing Bought it at 799€ from TechReview

Texto normalizado:
I love my new iPhone <NUM> ! 😍 Battery life is amazing Bought it at <EUR>799 from TechReview

== Verificación con Ejemplo Específico ==

Texto original:
🔥¡OFERTA! Compre 2x zapatos Nike a $99.99 (antes $150) 👟. ¡Válido hasta el 30/11/2023! Visita https://marketmind.com/oferta-nike. Atención @MariaP: ¿Envío gratis? 😃 #ModaDeportiva2023.

Texto normalizado (pipeline completo):
🔥 OFERTA! Compre <NUM> x zapatos Nike a <USD>99 <NUM> antes <USD>150 👟 Válido hasta el <NUM> / <NUM> / <NUM> ! Visita Atención Envío gratis? 😃 ModaDeportiva <NUM>


---

### **Fase 3: Normalización de Mayúsculas con Reconocimiento de Entidades**  
**Objetivo:** Convertir a minúsculas sin perder marcas comerciales o nombres de productos.  

#### **Requisitos:**  
1. **Preservar en mayúsculas:**  
   - Nombres de marcas (`Nike`, `iPhone`).  
   - Hashtags (`#ModaDeportiva2023`).  
   - Entidades geopolíticas (`Madrid`, `México`).  
2. **Convertir a minúsculas:**  
   - Verbo "compre" → "compre", "Zapatos" → "zapatos".  

#### **Pistas de Implementación:**  
- Usar `spaCy` con modelo `es_core_news_lg` para detectar entidades (ORG, LOC, PRODUCT).  
- Para marcas no reconocidas por spaCy (ej. `Zara`), cargar un diccionario personalizado desde un CSV.  

In [3]:
import spacy

# Cargar modelos de spaCy para español e inglés
try:
    nlp_es = spacy.load("es_core_news_md")
    nlp_en = spacy.load("en_core_web_md")
except OSError:
    print("Error: Los modelos de spaCy no están instalados.")
    print("Necesitas ejecutar la celda anterior para descargarlos.")

# Diccionario de marcas personalizadas que queremos preservar
# Podría ser cargado desde un CSV en un caso real
marcas_personalizadas = [
    "Nike", "Adidas", "Zara", "H&M", "Apple", "Samsung", "iPhone", 
    "Canon", "Sony", "Xiaomi", "MarketMind"
]

def normalize_case(text):
    """Función para normalizar mayúsculas preservando nombres de marcas, productos y entidades geopolíticas.
    
    Parámetros:
    text (str): Texto a normalizar (previamente limpiado y con números normalizados)
    
    Returns:
    str: Texto con mayúsculas normalizadas
    """
    # Primero detectamos el idioma para usar el modelo adecuado
    # Esta es una simplificación, para una aplicación real se usaría
    # un detector de idioma más robusto
    if any(word in text.lower() for word in ['el', 'la', 'los', 'las', 'en', 'con', 'para']):
        nlp = nlp_es
    else:
        nlp = nlp_en
        
    # Tokenizamos el texto
    doc = nlp(text)
    
    # Identificamos las entidades que queremos preservar
    entity_spans = []
    for ent in doc.ents:
        if ent.label_ in ["ORG", "PRODUCT", "LOC", "GPE"]:
            entity_spans.append((ent.start_char, ent.end_char, ent.text))
            
    # Identificamos hashtags y menciones
    tokens = text.split()
    for i, token in enumerate(tokens):
        if token.startswith("#"):
            start_idx = text.find(token)
            if start_idx >= 0:
                entity_spans.append((start_idx, start_idx + len(token), token))
    
    # Buscamos marcas personalizadas en el texto
    for marca in marcas_personalizadas:
        start = 0
        while True:
            start_idx = text.lower().find(marca.lower(), start)
            if start_idx < 0:
                break
            entity_spans.append((start_idx, start_idx + len(marca), marca))
            start = start_idx + len(marca)
            
    # Convertir todo a minúsculas primero
    text_lower = text.lower()
    
    # Reconstruir el texto preservando las entidades identificadas
    result = list(text_lower)
    for start, end, original in sorted(entity_spans, key=lambda x: x[0]):
        for i in range(start, end):
            if i < len(result):
                result[i] = text[i]
    
    return ''.join(result)

# Función para el pipeline completo: limpieza + normalización + casos
def process_text_full(text):
    """Aplica el pipeline completo de procesamiento de texto: 
    limpieza + normalización de números y unidades + normalización de mayúsculas.
    
    Parámetros:
    text (str): Texto crudo a procesar
    
    Returns:
    str: Texto completamente procesado
    """
    cleaned = clean_text(text)
    normalized = normalize_text(cleaned)
    case_normalized = normalize_case(normalized)
    return case_normalized

# Verificar con el ejemplo específico
ejemplo_especifico = "🔥¡OFERTA! Compre 2x zapatos Nike a $99.99 (antes $150) 👟. ¡Válido hasta el 30/11/2023! Visita https://marketmind.com/oferta-nike. Atención @MariaP: ¿Envío gratis? 😃 #ModaDeportiva2023."
print("== Verificación Pipeline Completo ==\n")
print("Texto original:")
print(ejemplo_especifico)

print("\nTexto limpio (Fase 1):")
cleaned = clean_text(ejemplo_especifico)
print(cleaned)

print("\nTexto normalizado (Fase 2):")
normalized = normalize_text(cleaned)
print(normalized)

print("\nTexto con mayúsculas normalizadas (Fase 3):")
case_normalized = normalize_case(normalized)
print(case_normalized)

print("\nTexto procesado (Pipeline completo):")
print(process_text_full(ejemplo_especifico))

== Verificación Pipeline Completo ==

Texto original:
🔥¡OFERTA! Compre 2x zapatos Nike a $99.99 (antes $150) 👟. ¡Válido hasta el 30/11/2023! Visita https://marketmind.com/oferta-nike. Atención @MariaP: ¿Envío gratis? 😃 #ModaDeportiva2023.

Texto limpio (Fase 1):
🔥 OFERTA! Compre 2 x zapatos Nike a $99 99 antes $150 👟 Válido hasta el 30 / 11 / 2023 ! Visita Atención Envío gratis? 😃 ModaDeportiva 2023

Texto normalizado (Fase 2):
🔥 OFERTA! Compre <NUM> x zapatos Nike a <USD>99 <NUM> antes <USD>150 👟 Válido hasta el <NUM> / <NUM> / <NUM> ! Visita Atención Envío gratis? 😃 ModaDeportiva <NUM>

Texto con mayúsculas normalizadas (Fase 3):
🔥 oferta! Compre <num> x zapatos Nike a <usd>99 <num> antes <USD>150 👟 válido hasta el <num> / <num> / <num> ! visita atención envío gratis? 😃 modadeportiva <num>

Texto procesado (Pipeline completo):
🔥 oferta! Compre <num> x zapatos Nike a <usd>99 <num> antes <USD>150 👟 válido hasta el <num> / <num> / <num> ! visita atención envío gratis? 😃 modadeportiva <n

In [4]:
# Probemos con algunos ejemplos aleatorios para verificar el funcionamiento completo del pipeline
for i in range(5):
    random_sample = random.choice(samples)
    ejemplo = random_sample['input']
    
    print(f"\n=== Ejemplo {i+1} ===")
    print("Texto original:")
    print(ejemplo)
    print("\nTexto procesado (pipeline completo):")
    print(process_text_full(ejemplo))
    print("-" * 80)


=== Ejemplo 1 ===
Texto original:
¿Dónde está mi pedido? Número de orden: #456DEF. Llevan 7 días sin actualizar.

Texto procesado (pipeline completo):
dónde está mi pedido? número de orden <num> def llevan <num> días sin actualizar
--------------------------------------------------------------------------------

=== Ejemplo 2 ===
Texto original:
Best headphones ever! Bought for 129.95 USD on 11/11/2023. 🎧

Texto procesado (pipeline completo):
best headphones ever! bought for <num> <usd>95 on <num> / <num> / <num> 🎧
--------------------------------------------------------------------------------

=== Ejemplo 3 ===
Texto original:
Achieved my fitness goal with this! Purchased 1x treadmill at $799.99. 🏃‍♂️

Texto procesado (pipeline completo):
achieved my fitness goal with this! purchased <num> x treadmill at <usd>799 <num> 🏃 ♂
--------------------------------------------------------------------------------

=== Ejemplo 4 ===
Texto original:
Me encanta este producto! Calidad superior, pe